In [1]:
SEED = 4444
_EPSILON = 1e-08

In [2]:
import sys
sys.path.append('../auton-survival/')
sys.path.append('../ddh/')

import pandas as pd
import numpy as np
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
import os
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import datetime

from ddh_torch import DynamicDeepHitTorch
import import_data as impt
from utils_eval import c_index, brier_score
from utils_helper import f_get_boosted_trainset
from utils_helper import loss_Log_Likelihood, loss_Ranking, loss_RNN_Prediction
from utils_helper import f_get_risk_predictions, _f_get_pred
from utils_helper import save_checkpoint, load_checkpoint

In [3]:
## for reproducibility
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [5]:
def model_eval(model, dataloader, device):
    model.eval()
    total_loss = 0
    Loss1, Loss2, Loss3 = 0, 0, 0

    for i, (b_data, b_data_mi, b_time, b_label, b_mask1, b_mask2, b_mask3) in enumerate(dataloader):
        b_data, _, b_time, b_label, b_mask1, b_mask2, b_mask3 = \
            b_data.float(), b_data_mi.float(), b_time.float(), b_label.float(), \
            b_mask1.float(), b_mask2.float(), b_mask3.float()
        b_data, b_time, b_label, b_mask1, b_mask2, b_mask3 = \
            b_data.to(device), b_time.to(device), b_label.to(device), \
            b_mask1.to(device), b_mask2.to(device), b_mask3.to(device)
        
        longitudinal_prediction, out = model(b_data)
        out = torch.concat([o.unsqueeze(1) for o in out], 1) # (B, num_Event, num_Category)
        loss1 = loss_Log_Likelihood(out, b_label, b_mask1, b_mask2)
        loss2 = loss_Ranking(out, b_time, b_label, b_mask3, num_Event, num_Category)
        loss3 = loss_RNN_Prediction(longitudinal_prediction, b_data)
        loss = alpha * loss1 + beta * loss2 + gamma * loss3
        
        total_loss += loss
        Loss1 += loss1
        Loss2 += loss2
        Loss3 += loss3
        
    return total_loss/len(dataloader), Loss1/len(dataloader), Loss2/len(dataloader), Loss3/len(dataloader)

### Import Dataset

In [9]:
data_mode                   = 'EEG_cr3' # competing risk with 3 causes
seed                        = SEED

##### IMPORT DATASET
'''
    num_Category            = max event/censoring time * 1.2
    num_Event               = number of evetns i.e. len(np.unique(label))-1
    max_length              = maximum number of measurements
    x_dim                   = data dimension including delta (1 + num_features)
    x_dim_cont              = dim of continuous features
    x_dim_bin               = dim of binary features
    mask1, mask2, mask3     = used for cause-specific network (FCNet structure)
'''

if data_mode == 'EEG_cr3':
    (x_dim, x_dim_cont, x_dim_bin), (data, time, label, time_original, time_to_last), \
        (mask1, mask2, mask3), (data_mi), trans_discrete_time = \
                    impt.import_dataset('../../eeg/competing-risk/EEG_processed_data_long_by_death_cat_exclude_3.csv')
    
    # This must be changed depending on the datasets, prediction/evaliation times of interest
    pred_time = [6, 12] # prediction time (in hours)
    eval_time = [12, 24,] # hours evaluation time (for C-index and Brier-Score)
else:
    print ('ERROR:  DATA_MODE NOT FOUND !!!')

_, num_Event, num_Category  = np.shape(mask1)  # dim of mask3: [subj, Num_Event, Num_Category]
max_length                  = np.shape(data)[1]


# file_path = '{}_{}'.format(data_mode, seed)

# if not os.path.exists(file_path):
#     os.makedirs(file_path)
    
num_Event, num_Category, max_length

/heinz-georgenas/anaconda3/envs/py39xiaobin/lib/python3.9/site-packages/pycox/preprocessing/label_transforms.py:113: UserWarning: Calling fit method, when 'cuts' are already defined. Leaving cuts unchanged.
  warnings.warn("Calling fit method, when 'cuts' are already defined. Leaving cuts unchanged.")


(3, 120, 12)

### Set Hyper-Parameters

In [11]:
from sklearn.model_selection import ParameterGrid
model_name = 'ddh_eeg_cr3'
param_grid = {'dropout': [0.2, 0.4], 
              'lr_train': [1e-4, 5e-4, 1e-3],
              'beta': [0.5, 1, 5],
              'gamma': [0.05, 0.1, 0.5],
             }
param_grid_list = list(ParameterGrid(param_grid))
print(len(param_grid_list))
param_grid_list

54


[{'beta': 0.5, 'dropout': 0.2, 'gamma': 0.05, 'lr_train': 0.0001},
 {'beta': 0.5, 'dropout': 0.2, 'gamma': 0.05, 'lr_train': 0.0005},
 {'beta': 0.5, 'dropout': 0.2, 'gamma': 0.05, 'lr_train': 0.001},
 {'beta': 0.5, 'dropout': 0.2, 'gamma': 0.1, 'lr_train': 0.0001},
 {'beta': 0.5, 'dropout': 0.2, 'gamma': 0.1, 'lr_train': 0.0005},
 {'beta': 0.5, 'dropout': 0.2, 'gamma': 0.1, 'lr_train': 0.001},
 {'beta': 0.5, 'dropout': 0.2, 'gamma': 0.5, 'lr_train': 0.0001},
 {'beta': 0.5, 'dropout': 0.2, 'gamma': 0.5, 'lr_train': 0.0005},
 {'beta': 0.5, 'dropout': 0.2, 'gamma': 0.5, 'lr_train': 0.001},
 {'beta': 0.5, 'dropout': 0.4, 'gamma': 0.05, 'lr_train': 0.0001},
 {'beta': 0.5, 'dropout': 0.4, 'gamma': 0.05, 'lr_train': 0.0005},
 {'beta': 0.5, 'dropout': 0.4, 'gamma': 0.05, 'lr_train': 0.001},
 {'beta': 0.5, 'dropout': 0.4, 'gamma': 0.1, 'lr_train': 0.0001},
 {'beta': 0.5, 'dropout': 0.4, 'gamma': 0.1, 'lr_train': 0.0005},
 {'beta': 0.5, 'dropout': 0.4, 'gamma': 0.1, 'lr_train': 0.001},
 {'beta':

In [12]:
boost_mode = 'ON' #{'ON', 'OFF'}

param = {'batch_size': 32,

         'num_epoch_burn_in': 20,
         'num_epoch': 100,

         'dropout': 0.2, # 1 - keep_prob
         'lr_train': 1e-4,

         'hidden_rnn': 64,
         'hidden_dim_FC': 64,
         'layers_rnn': 2,
         'hidden_att': 2,
         'hidden_cs' : 2,

         'alpha' :1.0,
         'beta'  :1,
         'gamma' :0.1
        }


batch_size = param['batch_size']
num_epoch  = param['num_epoch']

dropout    = param['dropout']
lr_train   = param['lr_train']

hidden_rnn = param['hidden_rnn']
hidden_dim_FC = param['hidden_dim_FC']
layers_rnn = param['layers_rnn']
hidden_att = param['hidden_att']
hidden_cs = param['hidden_cs']

alpha      = param['alpha']
beta       = param['beta']
gamma      = param['gamma']

device     = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


### Split Dataset into Train/Valid/Test Sets

In [13]:
### TRAINING-TESTING SPLIT
(tr_data,te_data, tr_data_mi, te_data_mi, tr_time,te_time, tr_label,te_label, tr_time_orinal,te_time_original,
 tr_time_to_last,te_time_to_last,
 tr_mask1,te_mask1, tr_mask2,te_mask2, tr_mask3,te_mask3) = \
        train_test_split(data, data_mi, time, label, time_original, time_to_last,
                         mask1, mask2, mask3, test_size=0.2, stratify=label, random_state=seed) 

(tr_data,va_data, tr_data_mi, va_data_mi, tr_time,va_time, tr_label,va_label, tr_time_orinal,va_time_original,
 tr_time_to_last, va_time_to_last,
 tr_mask1,va_mask1, tr_mask2,va_mask2, tr_mask3,va_mask3) = \
        train_test_split(tr_data, tr_data_mi, tr_time, tr_label, tr_time_orinal, tr_time_to_last,
                         tr_mask1, tr_mask2, tr_mask3, test_size=0.2, stratify=tr_label, random_state=seed) 

if boost_mode == 'ON':
    tr_data, tr_data_mi, tr_time, tr_label, tr_mask1, tr_mask2, tr_mask3 = \
            f_get_boosted_trainset(tr_data, tr_data_mi, tr_time, tr_label, tr_mask1, tr_mask2, tr_mask3)

In [15]:
tr_loader = DataLoader(list(zip(tr_data, tr_data_mi, tr_time, tr_label, tr_mask1, tr_mask2, tr_mask3)), 
                       batch_size=batch_size, shuffle=True)

va_loader = DataLoader(list(zip(va_data, va_data_mi, va_time, va_label, va_mask1, va_mask2, va_mask3)), 
                       batch_size=batch_size, shuffle=False)

te_loader = DataLoader(list(zip(te_data, te_data_mi, te_time, te_label, te_mask1, te_mask2, te_mask3)), 
                       batch_size=batch_size, shuffle=False)

### Train the Network

This implementation does not consider missing values.

In [16]:
num_Event

3

In [17]:
pred_time = [6, 12]
eval_time = [24, 48, 72]

In [ ]:
import pdb

Best_C_Index = 0


for params in tqdm(param_grid_list):
    dropout = params['dropout']
    lr_train = params['lr_train']
    beta = params['beta']
    gamma = params['gamma']
    print('='*80, '\n', params)

    model = \
        DynamicDeepHitTorch(input_dim  = x_dim,
                            output_dim = num_Category,
                            layers_rnn = param['layers_rnn'],
                            hidden_rnn = param['hidden_rnn'],
                            long_param = {'layers': 2*[param['hidden_dim_FC']], 'dropout': dropout}, 
                            att_param  = {'layers': param['hidden_att']*[param['hidden_dim_FC']], 'dropout': dropout}, 
                            cs_param   = {'layers': param['hidden_cs']*[param['hidden_dim_FC']], 'dropout': dropout},
                            risks      = num_Event
                           )
    optimizer_burn_in = torch.optim.Adam(model.parameters(), lr = lr_train)
    optimizer = torch.optim.Adam(model.parameters(), lr = lr_train)


    model.train()
    model.to(device)

    min_valid = 0.1
    early_stop_patience = 10
    ep = 0 # for tracking early stopping

    experiment_runtime = datetime.datetime.now().strftime('%Y%m%d_%H%M')
    tensorboard_string_extension = f"{model_name}_{SEED}"
    tensorboard_string_extension += f"_alpha{alpha}_beta{beta}_gamma{gamma}_BSZ{batch_size}_Lr{lr_train}"
    tensorboard_string_extension += f"_hiddenRNN{hidden_rnn}_hiddenFC{hidden_dim_FC}"
    tensorboard_string_extension += f"_layersRNN{layers_rnn}_hiddenAtt{hidden_att}_hiddenCS{hidden_cs}"
    tensorboard_string_extension += f"_dropout{dropout}"
    tensorboard_string_extension += "_" + experiment_runtime
    tensorboard_filename = tensorboard_string_extension
    writer = SummaryWriter(f"runs/{model_name}/{tensorboard_filename}")


    for epoch in range(param['num_epoch_burn_in']):
        model.train()
        for i, (b_data, b_data_mi, b_time, b_label, b_mask1, b_mask2, b_mask3) in enumerate(tr_loader):
            b_data, _, b_time, b_label, b_mask1, b_mask2, b_mask3 = \
                b_data.float(), b_data_mi.float(), b_time.float(), b_label.float(), \
                b_mask1.float(), b_mask2.float(), b_mask3.float()
            b_data, b_time, b_label, b_mask1, b_mask2, b_mask3 = \
                b_data.to(device), b_time.to(device), b_label.to(device), \
                b_mask1.to(device), b_mask2.to(device), b_mask3.to(device)

            longitudinal_prediction, out = model(b_data)
            out = torch.concat([o.unsqueeze(1) for o in out], 1) # (B, num_Event, num_Category)

            # loss1 = loss_Log_Likelihood(out, b_label, b_mask1, b_mask2)
            # loss2 = loss_Ranking(out, b_time, b_label, b_mask3, num_Event, num_Category)
            loss3 = loss_RNN_Prediction(longitudinal_prediction, b_data)
            tr_loss = loss3

            optimizer_burn_in.zero_grad()
            tr_loss.backward()
            optimizer_burn_in.step()


    for epoch in range(num_epoch):
        model.train()
        for i, (b_data, b_data_mi, b_time, b_label, b_mask1, b_mask2, b_mask3) in enumerate(tr_loader):
            b_data, _, b_time, b_label, b_mask1, b_mask2, b_mask3 = \
                b_data.float(), b_data_mi.float(), b_time.float(), b_label.float(), \
                b_mask1.float(), b_mask2.float(), b_mask3.float()
            b_data, b_time, b_label, b_mask1, b_mask2, b_mask3 = \
                b_data.to(device), b_time.to(device), b_label.to(device), \
                b_mask1.to(device), b_mask2.to(device), b_mask3.to(device)

            longitudinal_prediction, out = model(b_data)
            out = torch.concat([o.unsqueeze(1) for o in out], 1) # (B, num_Event, num_Category)

            loss1 = loss_Log_Likelihood(out, b_label, b_mask1, b_mask2)
            loss2 = loss_Ranking(out, b_time, b_label, b_mask3, num_Event, num_Category)
            loss3 = loss_RNN_Prediction(longitudinal_prediction, b_data)
            tr_loss = alpha * loss1 + beta * loss2 + gamma * loss3

            optimizer.zero_grad()
            tr_loss.backward()
            optimizer.step()

        model.eval()
        tr_loss, tr_loss1, tr_loss2, tr_loss3 = model_eval(model, tr_loader, device)
        va_loss, va_loss1, va_loss2, va_loss3 = model_eval(model, va_loader, device)

        ### VALIDATION  (based on average C-index of our interest)
        risk_all = f_get_risk_predictions(model, va_data, pred_time, eval_time, device)

        for p, p_time in enumerate(pred_time):
            pred_horizon = int(p_time)
            val_result1 = np.zeros([num_Event, len(eval_time)])

            for t, t_time in enumerate(eval_time):                
                eval_horizon = int(t_time) + pred_horizon
                for k in range(num_Event):
                    val_result1[k, t] = c_index(risk_all[k][:, p, t], va_time, (va_label[:,0] == k+1).astype(int), eval_horizon) #-1 for no event (not comparable)

            if p == 0:
                val_final1 = val_result1
            else:
                val_final1 = np.append(val_final1, val_result1, axis=0)

        tmp_valid = np.mean(val_final1)

        if tmp_valid >  min_valid:
            ep = 0
            min_valid = tmp_valid
            save_checkpoint(f"checkpoints/{model_name}/{writer.get_logdir().split('/')[-1]}_{SEED}.pt", model, optimizer, va_loss)
            print( 'updated.... average c-index = ' + str('%.4f' %(tmp_valid)))
        else:
            ep += 1  
            print("Counter {} of {}".format(ep, early_stop_patience))

        if ep >= early_stop_patience:
            print("Early stopping with best_cindex: {:.4f}".format(min_valid), 
                  "and val_cindex for this epoch: {:.4f}".format(tmp_valid))
            break

        # Tensorboard
        writer.add_scalar("Loss-Training/Loss", tr_loss, epoch)
        writer.add_scalar("Loss-Training/NLL Loss", tr_loss1, epoch)
        writer.add_scalar("Loss-Training/Ranking Loss", tr_loss2, epoch)
        writer.add_scalar("Loss-Training/LSTM MSELoss", tr_loss3, epoch)

        writer.add_scalar("Loss-Validation/Loss", va_loss, epoch)
        writer.add_scalar("Loss-Validation/NLL Loss", va_loss1, epoch)
        writer.add_scalar("Loss-Validation/Ranking Loss", va_loss2, epoch)
        writer.add_scalar("Loss-Validation/LSTM MSELoss", va_loss3, epoch)

        writer.add_scalar("C-index/Validation Avg C-index", min_valid, epoch)

        print('epoch {}, tr_loss: {:.4f}, va_loss: {:.4f}, va c-index: {:.4f}'.\
                      format(epoch+1, tr_loss, va_loss, tmp_valid))
        
    if min_valid > Best_C_Index:
        Best_C_Index = min_valid
        print('Better valudation c-index!!!')
        print(writer.get_logdir().split('/')[-1])

  0%|                                                                | 0/54 [00:00<?, ?it/s]

 {'beta': 0.5, 'dropout': 0.2, 'gamma': 0.05, 'lr_train': 0.0001}
updated.... average c-index = 0.6208
epoch 1, tr_loss: 31.4881, va_loss: 36.2923, va c-index: 0.6208
Counter 1 of 10
epoch 2, tr_loss: 31.3619, va_loss: 36.2717, va c-index: 0.6153
updated.... average c-index = 0.6245
epoch 3, tr_loss: 29.6774, va_loss: 35.9038, va c-index: 0.6245
updated.... average c-index = 0.6292
epoch 4, tr_loss: 29.3677, va_loss: 35.8742, va c-index: 0.6292
updated.... average c-index = 0.6346
epoch 5, tr_loss: 29.0159, va_loss: 35.6508, va c-index: 0.6346
updated.... average c-index = 0.6393
epoch 6, tr_loss: 28.3746, va_loss: 35.5731, va c-index: 0.6393
updated.... average c-index = 0.6479
epoch 7, tr_loss: 28.6759, va_loss: 35.6991, va c-index: 0.6479
updated.... average c-index = 0.6496
epoch 8, tr_loss: 28.1240, va_loss: 35.6501, va c-index: 0.6496
updated.... average c-index = 0.6568
epoch 9, tr_loss: 27.8617, va_loss: 35.6734, va c-index: 0.6568
Counter 1 of 10
epoch 10, tr_loss: 27.6488, va

  2%|▉                                                    | 1/54 [02:18<2:02:39, 138.86s/it]

Counter 10 of 10
Early stopping with best_cindex: 0.6840 and val_cindex for this epoch: 0.6767
Better valudation c-index!!!
ddh_eeg_cr3_4444_alpha1.0_beta0.5_gamma0.05_BSZ32_Lr0.0001_hiddenRNN64_hiddenFC64_layersRNN2_hiddenAtt2_hiddenCS2_dropout0.2_20230314_1025
 {'beta': 0.5, 'dropout': 0.2, 'gamma': 0.05, 'lr_train': 0.0005}
updated.... average c-index = 0.5652
epoch 1, tr_loss: 26.6152, va_loss: 39.4735, va c-index: 0.5652
updated.... average c-index = 0.6413
epoch 2, tr_loss: 23.4386, va_loss: 39.3555, va c-index: 0.6413
updated.... average c-index = 0.6473
epoch 3, tr_loss: 21.4149, va_loss: 38.4729, va c-index: 0.6473
updated.... average c-index = 0.6701
epoch 4, tr_loss: 20.1238, va_loss: 38.8832, va c-index: 0.6701
Counter 1 of 10
epoch 5, tr_loss: 20.2581, va_loss: 39.1446, va c-index: 0.6335
Counter 2 of 10
epoch 6, tr_loss: 22.6350, va_loss: 40.9687, va c-index: 0.6456
Counter 3 of 10
epoch 7, tr_loss: 22.8216, va_loss: 36.5716, va c-index: 0.6659
Counter 4 of 10
epoch 8, tr

  4%|█▉                                                   | 2/54 [03:57<1:39:46, 115.13s/it]

...

Counter 10 of 10
Early stopping with best_cindex: 0.5897 and val_cindex for this epoch: 0.5775
 {'beta': 1, 'dropout': 0.4, 'gamma': 0.05, 'lr_train': 0.0005}
updated.... average c-index = 0.6385
epoch 1, tr_loss: 33.8969, va_loss: 43.6298, va c-index: 0.6385
Counter 1 of 10
epoch 2, tr_loss: 29.7902, va_loss: 39.8372, va c-index: 0.6180
updated.... average c-index = 0.6395
epoch 3, tr_loss: 29.3989, va_loss: 39.5397, va c-index: 0.6395
Counter 1 of 10
epoch 4, tr_loss: 28.7069, va_loss: 38.8091, va c-index: 0.6378
Counter 2 of 10
epoch 5, tr_loss: 28.7981, va_loss: 39.5408, va c-index: 0.6034
Counter 3 of 10
epoch 6, tr_loss: 29.7172, va_loss: 40.8482, va c-index: 0.5708
Counter 4 of 10
epoch 7, tr_loss: 28.3005, va_loss: 39.7005, va c-index: 0.6088
Counter 5 of 10
epoch 8, tr_loss: 26.8548, va_loss: 39.4779, va c-index: 0.6112
Counter 6 of 10
epoch 9, tr_loss: 26.1046, va_loss: 41.2546, va c-index: 0.6291
Counter 7 of 10
epoch 10, tr_loss: 26.8416, va_loss: 42.5482, va c-index: 0.600

 54%|██████████████████████████▊                       | 29/54 [1:21:45<1:09:29, 166.76s/it]

Counter 10 of 10
Early stopping with best_cindex: 0.7022 and val_cindex for this epoch: 0.6810
 {'beta': 1, 'dropout': 0.4, 'gamma': 0.05, 'lr_train': 0.001}
updated.... average c-index = 0.5753
epoch 1, tr_loss: 37.3326, va_loss: 42.7602, va c-index: 0.5753
updated.... average c-index = 0.6328
epoch 2, tr_loss: 34.3897, va_loss: 40.4844, va c-index: 0.6328
Counter 1 of 10
epoch 3, tr_loss: 34.1918, va_loss: 39.5611, va c-index: 0.5941
Counter 2 of 10
epoch 4, tr_loss: 31.7399, va_loss: 38.6338, va c-index: 0.6258
Counter 3 of 10
epoch 5, tr_loss: 32.0524, va_loss: 38.6383, va c-index: 0.5962
Counter 4 of 10
epoch 6, tr_loss: 31.2655, va_loss: 38.1040, va c-index: 0.6324
updated.... average c-index = 0.6347
epoch 7, tr_loss: 31.5855, va_loss: 37.9615, va c-index: 0.6347
Counter 1 of 10
epoch 8, tr_loss: 31.9071, va_loss: 38.4120, va c-index: 0.6308
updated.... average c-index = 0.6569
epoch 9, tr_loss: 29.9938, va_loss: 37.4148, va c-index: 0.6569
updated.... average c-index = 0.6761
e

 56%|███████████████████████████▊                      | 30/54 [1:23:51<1:01:52, 154.67s/it]

Counter 10 of 10
Early stopping with best_cindex: 0.6811 and val_cindex for this epoch: 0.2930
 {'beta': 1, 'dropout': 0.4, 'gamma': 0.1, 'lr_train': 0.0001}
updated.... average c-index = 0.5018
epoch 1, tr_loss: 61.0642, va_loss: 67.7218, va c-index: 0.5018
updated.... average c-index = 0.5414
epoch 2, tr_loss: 56.8409, va_loss: 67.2079, va c-index: 0.5414
Counter 1 of 10
epoch 3, tr_loss: 56.2519, va_loss: 66.4977, va c-index: 0.5386
updated.... average c-index = 0.5494
epoch 4, tr_loss: 54.9938, va_loss: 66.2892, va c-index: 0.5494
updated.... average c-index = 0.5577
epoch 5, tr_loss: 54.6680, va_loss: 65.8139, va c-index: 0.5577
updated.... average c-index = 0.5586
epoch 6, tr_loss: 54.3779, va_loss: 65.5380, va c-index: 0.5586
Counter 1 of 10
epoch 7, tr_loss: 54.9518, va_loss: 65.8667, va c-index: 0.5553
updated.... average c-index = 0.5610
epoch 8, tr_loss: 53.9554, va_loss: 66.1899, va c-index: 0.5610
Counter 1 of 10
epoch 9, tr_loss: 54.2507, va_loss: 66.6523, va c-index: 0.5

In [19]:
model

DynamicDeepHitTorch(
  (embedding): LSTM(116, 64, num_layers=2, bias=False, batch_first=True)
  (longitudinal): Sequential(
    (0): Dropout(p=0.4, inplace=False)
    (1): Linear(in_features=64, out_features=64, bias=True)
    (2): ReLU()
    (3): Linear(in_features=64, out_features=64, bias=True)
    (4): ReLU()
    (5): Linear(in_features=64, out_features=116, bias=True)
  )
  (attention): Sequential(
    (0): Dropout(p=0.4, inplace=False)
    (1): Linear(in_features=180, out_features=64, bias=True)
    (2): ReLU()
    (3): Linear(in_features=64, out_features=64, bias=True)
    (4): ReLU()
    (5): Linear(in_features=64, out_features=1, bias=True)
  )
  (attention_soft): Softmax(dim=1)
  (cause_specific): ModuleList(
    (0): Sequential(
      (0): Dropout(p=0.4, inplace=False)
      (1): Linear(in_features=180, out_features=64, bias=True)
      (2): ReLU()
      (3): Linear(in_features=64, out_features=64, bias=True)
      (4): ReLU()
      (5): Linear(in_features=64, out_features=1

### Test the Trained Network
see the code in `eval-ddh.ipynb` file